In [1]:
# Use tf1 for code tf v1.x
import tensorflow.compat.v1 as tf
import numpy as np
from sklearn.datasets import load_boston

%matplotlib inline
%reload_ext tensorboard
# tf1.disable_v2_behavior()

In [2]:
boston = load_boston()
features = np.array(boston.data)
labels = np.array(boston.target)
n_training_samples = features.shape[0]
n_dim = features.shape[1]
print('The dataset has',n_training_samples,'training samples.')
print('The dataset has',n_dim,'features.')

The dataset has 506 training samples.
The dataset has 13 features.


In [3]:
def normalize(dataset):
    mu = np.mean(dataset, axis = 0)
    sigma = np.std(dataset, axis = 0)
    return (dataset-mu)/sigma

In [4]:
features_norm = normalize(features)
train_x = np.transpose(features_norm)
train_y = np.transpose(labels)
print(train_x.shape)
print(train_y.shape)

(13, 506)
(506,)


In [5]:
train_y = train_y.reshape(1,len(train_y))
print(train_y.shape)

(1, 506)


In [6]:
! mkdir -p logs/linreg
logdir = "logs/linreg"
# Prepare the computational graph
tf.disable_v2_behavior()
tf.reset_default_graph()
X = tf.placeholder(tf.float32, [n_dim, None])
Y = tf.placeholder(tf.float32, [1, None])
learning_rate = tf.placeholder(tf.float32, shape=())
W = tf.Variable(tf.ones([n_dim,1]))
b = tf.Variable(tf.zeros(1))

init = tf.global_variables_initializer()
y_ = tf.matmul(tf.transpose(W),X)+b
cost = tf.reduce_mean(tf.square(y_-Y))
training_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

Instructions for updating:
non-resource variables are not supported in the long term


In [11]:
def run_linear_model(learning_r, training_epochs, train_obs, train_labels, debug = False):
    sess = tf.Session()
    sess.run(init)
    writer = tf.summary.FileWriter(logdir, sess.graph)
    cost_history = np.empty(shape=[0], dtype = float)
    cost_ = sess.run(cost, feed_dict={ X:train_obs, Y: train_labels, learning_rate: learning_r})
    tf.summary.scalar('loss', cost_)
    merged = tf.summary.merge_all()
    for epoch in range(training_epochs+1):
        sess.run(training_step, feed_dict = {X: train_obs, Y: train_labels, learning_rate: learning_r})
        cost_ = sess.run(cost, feed_dict={ X:train_obs, Y: train_labels, learning_rate: learning_r})
        cost_history = np.append(cost_history, cost_)
        if (epoch % 1000 == 0) & debug:
            summary = sess.run(merged)
            writer.add_summary(summary, epoch)
            writer.flush()
            print("Reached epoch",epoch,"cost J =", str.format('{0:.6f}', cost_))
    return sess, cost_history

In [12]:
sess, cost_history = run_linear_model(learning_r = 0.01,
                                      training_epochs = 10000,
                                      train_obs = train_x,
                                      train_labels = train_y,
                                      debug = True)

%tensorboard --logdir logs/linreg

Reached epoch 0 cost J = 606.887146
Reached epoch 1000 cost J = 21.952604
Reached epoch 2000 cost J = 21.899084
Reached epoch 3000 cost J = 21.895168
Reached epoch 4000 cost J = 21.894857
Reached epoch 5000 cost J = 21.894833
Reached epoch 6000 cost J = 21.894831
Reached epoch 7000 cost J = 21.894833
Reached epoch 8000 cost J = 21.894831
Reached epoch 9000 cost J = 21.894833
Reached epoch 10000 cost J = 21.894833


Reusing TensorBoard on port 6006 (pid 18555), started 1:13:21 ago. (Use '!kill 18555' to kill it.)

In [9]:
# close tensorboard server
! pids=`ps -ef|grep tensorboard|awk '{print $2}'`; for p in $pids; do kill -INT $p; done